In [74]:
import ase
from utils import *

In [17]:
Atomsl = ase.io.read('first_trials/coronene.990000.cfg')
Atomse = ase.io.read('first_trials/coronene.10000.cfg')
Atomsm = ase.io.read('first_trials/coronene.500000.cfg')

In [75]:
sp2 = []  # number of carbon with sp2-hybr; each step
carbon_cycles = []  # number of carbon cycles with sizes 4,5,6,7; each step
cluster_num = []  # number of molecules with size greater than half of coronene; each step

In [ ]:
for step in range(0, 11140000, 10000):
    atoms = ase.io.read('data/first_trials/coronene.%d.cfg' % d)
    _, sp2_atoms, _, _ = get_carbon_hybr(atoms)
    sp2.append(sp2_atoms.shape[0])
    cycle_num = 0
    for cycle_length in range(4,8):
        cycle_num += len(get_carbon_cycles(atoms, cycle_length))
    carbon_cycles.append(cycle_num)
    cluster.append(get_clusters_num(atoms, ))

In [18]:
arr = Atomsl.get_positions()
arr = np.broadcast_to(arr, (arr.shape[0], arr.shape[0], arr.shape[1]))
arr = np.sum(arr**2, axis=2)**0.5
arr.shape

(576, 576)

In [19]:
def get_bond_graph(atoms, bond_length=1.85):
    coords = atoms.get_positions()
    coords = np.broadcast_to(coords, (coords.shape[0], coords.shape[0], coords.shape[1]))
    distances = coords - np.transpose(coords, (1, 0, 2))
    del coords
    box_params = np.broadcast_to(atoms.get_cell().lengths(), distances.shape)
    distances = np.minimum(np.abs(distances), np.abs(box_params - np.abs(distances)))
    del box_params
    distances = np.sum(distances**2, axis=len(distances.shape)-1)**0.5
    adj_matrix = distances
    del distances
    adj_matrix[adj_matrix > bond_length] = 0
    bond_graph = nx.from_numpy_matrix(adj_matrix)
    
    return bond_graph    

In [20]:
view(Atomse, viewer='ngl')

In [21]:
bond_graph = get_bond_graph(Atoms)

NameError: name 'Atoms' is not defined

In [ ]:
nx.algorithms.components.number_connected_components(bond_graph)

In [22]:
np.count_nonzero(Atoms.get_atomic_numbers()==6)

NameError: name 'Atoms' is not defined

In [23]:
def get_atom_idxs(atoms, atom_type):
    return np.nonzero(atoms.get_atomic_numbers()==atom_type)[0]

In [24]:
get_atom_idxs(Atoms, atom_type=6)

NameError: name 'Atoms' is not defined

In [25]:
bond_graph = get_bond_graph(Atoms)
len(list(bond_graph.neighbors(2)))
get_atom_idxs(Atoms, atom_type=6)

NameError: name 'Atoms' is not defined

In [26]:
def calc_carb_hybr(atoms):
    bond_graph = get_bond_graph(atoms)
    carbon_idxs = get_atom_idxs(atoms, atom_type=6)
    print(type(carbon_idxs))
    sp1 = []
    sp2 = []
    sp3 = []
    others = []
    for carbon in carbon_idxs:
        if len(list(bond_graph.neighbors(carbon))) == 2: sp1.append(carbon)
        elif len(list(bond_graph.neighbors(carbon))) == 3: sp2.append(carbon)
        elif len(list(bond_graph.neighbors(carbon))) == 4: sp3.append(carbon)
        else: others.append(carbon)
    
    sp1 = np.array(sp1)
    sp2 = np.array(sp2)
    sp3 = np.array(sp3)
    others = np.array(others)
    
    return (sp1, sp2, sp3, others)

In [27]:
calc_carb_hybr(Atomse)

NameError: name 'nx' is not defined

In [28]:
hyb = []
for i in range(len(Atoms.get_atomic_numbers())):
    if Atoms.get_atomic_numbers()[i] == 6:
        count = 0
        for j in range(len(Atoms.get_atomic_numbers())):
               x0 = Atoms.get_positions()[i][0]
               y0 = Atoms.get_positions()[i][1]
               z0 = Atoms.get_positions()[i][2]
               x = Atoms.get_positions()[j][0]
               y = Atoms.get_positions()[j][1]
               z = Atoms.get_positions()[j][2]
               if in_cube(x0,y0,z0,x,y,z,1.8) == True:
                   r = np.sqrt((x-x0)**2 + (y-y0)**2 + (z-z0)**2)
                   if r<=1.8:
                       count+=1
            
        hyb.append([i,count-1])
    

NameError: name 'Atoms' is not defined

In [29]:
def return_hybridisation(Atoms):
    Dist = Atoms.get_all_distances(mic=True)
    hybr = []
    for i in range(len(Atoms.get_atomic_numbers())):
        if Atoms.get_atomic_numbers()[i] == 6:
            count = 0
            for j in range(len(Dist[i])):
                if Dist[i][j] < 1.8:
                    count += 1
            hybr.append([i,count-1])
    for a in range(len(hybr)):
        if hybr[a][1] == 0:
            hybr[a][1] = 'no_hybr'
        elif hybr[a][1] == 1:
            hybr[a][1] = 'no_hybr'
        elif hybr[a][1] == 2:
            hybr[a][1] = 'sp'
        elif hybr[a][1] == 3:
            hybr[a][1] = 'sp2'
        elif hybr[a][1] ==  4:
            hybr[a][1] = 'sp3'
        else: 
            hybr[a][1] = '???'
    return hybr
    

In [30]:
early = return_hybridisation(Atomse)
mid = return_hybridisation(Atoms)
late = return_hybridisation(Atomsl)

NameError: name 'Atoms' is not defined

In [31]:
early

[[1, 'sp2'],
 [3, 'sp2'],
 [5, 'sp2'],
 [6, 'sp2'],
 [7, 'sp2'],
 [8, 'sp2'],
 [9, 'sp2'],
 [11, 'sp2'],
 [12, 'sp2'],
 [14, 'sp2'],
 [15, 'sp2'],
 [16, 'sp2'],
 [17, 'sp2'],
 [18, 'sp2'],
 [19, 'sp2'],
 [20, 'sp2'],
 [22, 'sp2'],
 [25, 'sp2'],
 [27, 'sp2'],
 [28, 'sp2'],
 [29, 'sp2'],
 [30, 'sp2'],
 [31, 'sp2'],
 [32, 'sp2'],
 [34, 'sp2'],
 [36, 'sp2'],
 [37, 'sp2'],
 [38, 'sp2'],
 [40, 'sp2'],
 [44, 'sp2'],
 [45, 'sp2'],
 [46, 'sp2'],
 [48, 'sp2'],
 [49, 'sp2'],
 [51, 'sp2'],
 [52, 'sp2'],
 [53, 'sp2'],
 [56, 'sp2'],
 [57, 'sp2'],
 [58, 'sp2'],
 [59, 'sp2'],
 [60, 'sp2'],
 [61, 'sp2'],
 [62, 'sp2'],
 [63, 'sp2'],
 [64, 'sp2'],
 [65, 'sp2'],
 [67, 'sp2'],
 [69, 'sp2'],
 [70, 'sp2'],
 [71, 'sp2'],
 [73, 'sp2'],
 [74, 'sp2'],
 [76, 'sp2'],
 [78, 'sp2'],
 [79, 'sp2'],
 [85, 'sp2'],
 [86, 'sp2'],
 [87, 'sp2'],
 [88, 'sp2'],
 [89, 'sp2'],
 [93, 'sp2'],
 [94, 'sp2'],
 [97, 'sp2'],
 [98, 'sp2'],
 [99, 'sp2'],
 [100, 'sp2'],
 [101, 'sp2'],
 [102, 'sp2'],
 [103, 'sp2'],
 [104, 'sp2'],
 [105, '

In [32]:
late

NameError: name 'late' is not defined

In [57]:
def get_clusters_num(atoms, min_cluster_size=5):
    bond_graph = get_bond_graph(atoms)
    cluster_sizes = np.array([len(comp) for comp in nx.connected_components(bond_graph)])
    return np.nonzero(cluster_sizes >= min_cluster_size)[0].shape[0]

In [58]:
get_clusters_num(Atomse)

16

In [64]:
G = nx.Graph()
nx.add_path(G, [0,1,2,5,4,3,0])
G.add_edge(1, 4)
G.add_edge(4, 7)
nx.add_path(G, [3, 6, 7, 8, 5])

In [70]:
DiG = G.to_directed()
nx.algorithms.cycles.minimum_cycle_basis(G)

list

In [73]:
def get_carbon_cycles(atoms, cycle_length=6):
    bond_graph = get_bond_graph(atoms)
    cycle_list = []
    for cycle in nx.algorithms.cycles.minimum_cycle_basis(bond_graph):
        if len(cycle) == cycle_length: 
            cycle_list.append(cycle)
    return cycle_list
len(get_carbon_cycles(Atomse))

112